In [1]:
# third-party
import pandas as pd
import numpy as np
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_fr_detection_performance, get_breath_parameters_performance
from delay_performance import normality_test, plot_inspiration_vs_expiration
from constants import CATEGORICAL_PALETTE


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]


### Activity-specific only FR analysis

In [3]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [4]:
performance_all_participants = get_fr_detection_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNB,MAG,1.02,0.90,0.92,0.03 $\pm$ 0.33,0 %
1,SNB,PZT,1.13,0.56,0.63,0.05 $\pm$ 0.63,0 %
2,SGB,MAG,1.01,0.77,0.78,0.23 $\pm$ 0.92,2 %
3,SGB,PZT,1.58,0.35,0.56,1.14 $\pm$ 1.93,11 %
4,MIXB,MAG,0.98,0.91,0.89,0.0 $\pm$ 0.23,0 %
5,MIXB,PZT,1.12,0.47,0.53,0.22 $\pm$ 0.86,4 %
6,STNB,MAG,1.04,0.89,0.92,0.0 $\pm$ 0.26,0 %
7,STNB,PZT,1.04,0.55,0.57,0.26 $\pm$ 0.87,6 %
8,MCH,MAG,1.16,0.72,0.83,0.05 $\pm$ 0.42,1 %
9,MCH,PZT,1.61,0.46,0.74,0.4 $\pm$ 0.67,10 %


In [5]:
mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True) 
pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True)

a = mag / pzt 
a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
a

,Ratio,Precision,Recall,Activity
0,0.902655,1.607143,1.460317,SNB
1,0.639241,2.200000,1.392857,SGB
2,0.875000,1.936170,1.679245,MIXB
3,1.000000,1.618182,1.614035,STNB
4,0.720497,1.565217,1.121622,MCH
5,0.770833,1.702128,1.308824,SQT
6,0.849206,1.438596,1.208333,AAL
7,0.808000,1.451613,1.166667,AAR
8,0.766423,1.509434,1.136986,ALL
9,0.789062,1.490909,1.185714,ALR


In [6]:
mag["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
pzt["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].reset_index(drop=True)["Activity"]
mag

,Ratio,Precision,Recall,Activity
0,1.02,0.90,0.92,SNB
1,1.01,0.77,0.78,SGB
2,0.98,0.91,0.89,MIXB
3,1.04,0.89,0.92,STNB
4,1.16,0.72,0.83,MCH
5,1.11,0.80,0.89,SQT
6,1.07,0.82,0.87,AAL
7,1.01,0.90,0.91,AAR
8,1.05,0.80,0.83,ALL
9,1.01,0.82,0.83,ALR


### Participant-specific only FR analysis

In [7]:
# Transform overview from participant specific into all participants
overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [8]:
get_fr_detection_performance(overview_all_activities, target="ID")

,ID,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,7OYX,MAG,1.09,0.82,0.90,0.04 $\pm$ 0.35,0 %
1,7OYX,PZT,1.89,0.39,0.74,0.76 $\pm$ 0.88,15 %
2,NO15,MAG,1.13,0.78,0.88,-0.0 $\pm$ 0.35,0 %
3,NO15,PZT,1.52,0.39,0.59,0.34 $\pm$ 0.7,7 %
4,G8B7,MAG,1.13,0.55,0.62,0.17 $\pm$ 0.5,5 %
5,G8B7,PZT,1.06,0.47,0.50,-0.02 $\pm$ 0.48,-1 %
6,EPE2,MAG,0.98,0.92,0.90,0.24 $\pm$ 0.46,5 %
7,EPE2,PZT,1.36,0.47,0.63,0.46 $\pm$ 0.95,9 %
8,HAK8,MAG,1.11,0.79,0.87,0.03 $\pm$ 0.35,1 %
9,HAK8,PZT,1.44,0.58,0.84,0.58 $\pm$ 0.6,17 %


### Overall FR Analysis

In [9]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [10]:
get_fr_detection_performance(overview_all, target=None)

,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,MAG,1.10,0.77,0.85,0.08 $\pm$ 0.41,NaN
1,PZT,1.36,0.52,0.71,0.41 $\pm$ 0.73,NaN


In [11]:
fr_detection_all = get_fr_detection_performance(overview, target="both")
fr_detection_all

,ID,Activity,Sensor,Ratio,Precision,Recall
0,7OYX,SNB,MAG,1.00,0.93,0.93
1,7OYX,SNB,PZT,1.07,0.23,0.24
2,7OYX,SGB,MAG,1.00,1.00,1.00
3,7OYX,SGB,PZT,1.31,0.29,0.38
4,7OYX,MIXB,MAG,1.00,1.00,1.00
...,...,...,...,...,...,...
473,D4GQ,UAR,PZT,1.00,0.29,0.29
474,D4GQ,SE,MAG,1.33,0.71,0.95
475,D4GQ,SE,PZT,1.14,0.42,0.48
476,D4GQ,SS,MAG,3.00,0.33,1.00


In [12]:
fr_mag = fr_detection_all[fr_detection_all["Sensor"]=="MAG"]
fr_pzt = fr_detection_all[fr_detection_all["Sensor"]=="PZT"]

In [13]:
metric = "Ratio"
normality_test(fr_mag[metric], sensor=metric, type="All data")
normality_test(fr_pzt[metric], sensor=metric, type="All data")

In [14]:
metric = "Precision"
normality_test(fr_mag[metric], sensor=metric, type="All data")
normality_test(fr_pzt[metric], sensor=metric, type="All data")

In [15]:
metric = "Recall"
normality_test(fr_mag[metric], sensor=metric, type="All data")
normality_test(fr_pzt[metric], sensor=metric, type="All data")

In [16]:
print(f'Ratio: {scipy.stats.wilcoxon(fr_mag["Ratio"], fr_pzt["Ratio"])}')
print(f'Precision: {scipy.stats.wilcoxon(fr_mag["Precision"], fr_pzt["Precision"])}')
print(f'Recall: {scipy.stats.wilcoxon(fr_mag["Recall"], fr_pzt["Recall"])}')

Ratio: WilcoxonResult(statistic=3603.5, pvalue=1.7440390766421865e-16)
Precision: WilcoxonResult(statistic=1326.0, pvalue=2.816423876488204e-33)
Recall: WilcoxonResult(statistic=4824.0, pvalue=2.927170083719441e-13)


In [17]:
wilc = pd.DataFrame(columns=["Activity", "Metric", "Statistic", "p-value"])

for metric in ["Ratio", "Precision", "Recall"]:
    for activity in mag["Activity"].unique():
        new_entry = {}
        scipy.stats.wilcoxon(fr_mag[fr_mag["Activity"]==activity][metric], fr_pzt[fr_pzt["Activity"]==activity][metric])
        new_entry["Activity"] = activity
        new_entry["Metric"] = metric
        new_entry["Statistic"], new_entry["p-value"] = scipy.stats.wilcoxon(fr_mag[fr_mag["Activity"]==activity][metric].values, fr_pzt[fr_pzt["Activity"]==activity][metric].values)
        wilc.loc[len(wilc)] = new_entry
        
wilc

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4102: UserWarning:

Sample size too small for normal approximation.

/Users/anasofiacc/dev/

,Activity,Metric,Statistic,p-value
0,SNB,Ratio,30.5,0.166730
1,SGB,Ratio,11.0,0.009181
2,MIXB,Ratio,8.0,0.008672
3,STNB,Ratio,13.5,0.527599
4,MCH,Ratio,24.0,0.021393
5,SQT,Ratio,1.5,0.000888
6,AAL,Ratio,19.5,0.038205
7,AAR,Ratio,1.0,0.006910
8,ALL,Ratio,16.0,0.012453
9,ALR,Ratio,30.5,0.050659


In [18]:
#wilc[wilc["Metric"]=="Ratio"][wilc["p-value"] < 0.05].sort_values(by="Statistic")
wilc[wilc["Activity"]=="SGB"]

,Activity,Metric,Statistic,p-value
1,SGB,Ratio,11.0,0.009181
16,SGB,Precision,0.0,0.000031
31,SGB,Recall,12.0,0.010617


In [19]:
9.58e-3

0.00958

### Overall delay analysis

In [20]:
delays_i_all_mag, delays_e_all_mag = [], []
delays_i_all_pzt, delays_e_all_pzt = [], []

for activity in overview_all_participants.keys():

    delays_i_all_mag += overview_all_participants[activity]["MAG"]["delay_i"] 
    delays_e_all_mag += overview_all_participants[activity]["MAG"]["delay_e"]

    delays_i_all_pzt += overview_all_participants[activity]["PZT"]["delay_i"]
    delays_e_all_pzt += overview_all_participants[activity]["PZT"]["delay_e"]

In [21]:
print(f"min delay mag {min(delays_i_all_mag + delays_e_all_mag, key=abs)}")
print(f"min delay pzt {min(delays_i_all_pzt + delays_e_all_pzt, key=abs)}")

print(f"max delay mag {max(delays_i_all_mag + delays_e_all_mag, key=abs)}")
print(f"max delay pzt {max(delays_i_all_pzt + delays_e_all_pzt, key=abs)}")

print(f"mean delay mag {round(np.mean(delays_i_all_mag + delays_e_all_mag), 2)} +/- {np.round(np.std(delays_i_all_mag + delays_e_all_mag), 2)}")
print(f"mean delay pzt {np.round(np.mean(delays_i_all_pzt + delays_e_all_pzt), 2)} +/- {np.round(np.std(delays_i_all_pzt + delays_e_all_pzt), 2)}")

min delay mag -0.0
min delay pzt -0.0
max delay mag 8.15
max delay pzt 8.77
mean delay mag 0.08 +/- 0.41
mean delay pzt 0.41 +/- 0.73


In [22]:
normality_test(delays_i_all_mag + delays_e_all_mag, sensor="MAG", type="inspiration+expiration")
normality_test(delays_i_all_pzt + delays_e_all_pzt, sensor="PZT", type="inspiration+expiration")

### Event-specific delay analysis

In [23]:
normality_test(delays_i_all_mag, sensor="MAG", type="inspiration")
normality_test(delays_e_all_mag, sensor="MAG", type="expiration")
normality_test(delays_i_all_pzt, sensor="PZT", type="inspiration", categorical_palette=CATEGORICAL_PALETTE[2:])
normality_test(delays_e_all_pzt, sensor="PZT", type="expiration", categorical_palette=CATEGORICAL_PALETTE[2:])

In [24]:
delays_df = pd.DataFrame(columns=["Sensor", "Type", "Delay"])
delays_df["Delay"] = delays_i_all_mag + delays_e_all_mag + delays_i_all_pzt + delays_e_all_pzt
delays_df["Sensor"] = ["MAG"] * len(delays_i_all_mag + delays_e_all_mag) + ["PZT"] * len(delays_i_all_pzt + delays_e_all_pzt)
delays_df["Type"] = ["Inhalation"] * len(delays_i_all_mag) + ["Exhalation"] * len(delays_e_all_mag) + ["Inhalation"] * len(delays_i_all_pzt) + ["Exhalation"] * len(delays_e_all_pzt)
delays_df


,Sensor,Type,Delay
0,MAG,Inhalation,-0.22
1,MAG,Inhalation,-0.05
2,MAG,Inhalation,-0.11
3,MAG,Inhalation,0.01
4,MAG,Inhalation,0.12
...,...,...,...
11204,PZT,Exhalation,-0.06
11205,PZT,Exhalation,2.53
11206,PZT,Exhalation,2.98
11207,PZT,Exhalation,-0.28


In [25]:
plot_inspiration_vs_expiration(delays_df)

#### Statistical testing: comparing inhalation vs expiration

In [26]:
print(scipy.stats.mannwhitneyu(delays_i_all_mag, delays_e_all_mag))
print(scipy.stats.mannwhitneyu(delays_i_all_pzt, delays_e_all_pzt))

MannwhitneyuResult(statistic=5095677.5, pvalue=4.720863550489277e-10)
MannwhitneyuResult(statistic=3370589.5, pvalue=0.011034065147345787)


In [27]:
print(len(delays_i_all_mag + delays_e_all_mag))
print(len(delays_i_all_pzt + delays_e_all_pzt))

6118
5091


### Correlation between strap-band perimeter and FR performance

In [28]:
strap_thorax_ratio_dict = {
    '7OYX': 0.94,
    'NO15': 0.99,
    'G8B7': 0.96,
    'EPE2': 0.93,
    'HAK8': 0.94,
    '1BST': 0.98,
    '83J1': 0.97,
    'QMQ7': 0.96,
    '9TUL': 0.9,
    'FTD7': 0.9,
    'Y6O3': 0.92,
    '2QWT': 0.85,
    'F9AF': 0.87,
    'P4W9': 0.96,
    'W8Z9': 0.94,
    'D4GQ': 0.92
}

In [29]:
strap_thorax_performance_df = get_fr_detection_performance(overview, target="both")
strap_thorax_performance_df = strap_thorax_performance_df[strap_thorax_performance_df["Sensor"] == "MAG"]
strap_thorax_performance_df['strap-to-thorax ratio'] = strap_thorax_performance_df["ID"].apply(lambda id: strap_thorax_ratio_dict[id])
strap_thorax_performance_df


,ID,Activity,Sensor,Ratio,Precision,Recall,strap-to-thorax ratio
0,7OYX,SNB,MAG,1.00,0.93,0.93,0.94
2,7OYX,SGB,MAG,1.00,1.00,1.00,0.94
4,7OYX,MIXB,MAG,1.00,1.00,1.00,0.94
6,7OYX,STNB,MAG,1.00,0.95,0.95,0.94
8,7OYX,MCH,MAG,0.95,0.81,0.77,0.94
...,...,...,...,...,...,...,...
468,D4GQ,ALR,MAG,0.58,0.79,0.46,0.92
470,D4GQ,UAL,MAG,2.55,0.39,1.00,0.92
472,D4GQ,UAR,MAG,1.05,0.82,0.86,0.92
474,D4GQ,SE,MAG,1.33,0.71,0.95,0.92


In [30]:
scipy.stats.spearmanr(strap_thorax_performance_df["Ratio"].values, strap_thorax_performance_df["strap-to-thorax ratio"].values)

SignificanceResult(statistic=0.09083857081488336, pvalue=0.16155421305653497)

### Analysis on alternative positioning

In [31]:
overview_middle_all_participants = transform_overview_on_target(overview_middle, target="Activity")
performance_middle_all_participants = get_fr_detection_performance(overview_middle_all_participants, target="Activity")
performance_middle_all_participants = performance_middle_all_participants[performance_middle_all_participants["Sensor"]=="MAG"]
performance_middle_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNBm,MAG,1.00,0.93,0.93,0.05 $\pm$ 0.32,0 %
2,UALm,MAG,1.26,0.72,0.91,0.31 $\pm$ 0.5,8 %
4,UARm,MAG,1.36,0.67,0.91,0.36 $\pm$ 0.51,8 %


In [32]:
performance_middle_vs_left = pd.DataFrame(columns=performance_middle_all_participants.columns)

for activity in performance_middle_all_participants["Activity"].unique():
    new_entry = performance_all_participants[(performance_all_participants["Activity"]==activity[:-1]) & (performance_all_participants["Sensor"]=="MAG")]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

    new_entry = performance_middle_all_participants[performance_middle_all_participants["Activity"]==activity]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

performance_middle_vs_left.drop(columns=["Sensor", "Mean absolute delay $\pm$ SD", "Adjusted delay"], inplace=True)
performance_middle_vs_left

,Activity,Ratio,Precision,Recall
0,SNB,1.02,0.90,0.92
1,SNBm,1.00,0.93,0.93
2,UAL,1.17,0.71,0.84
3,UALm,1.26,0.72,0.91
4,UAR,1.08,0.86,0.92
5,UARm,1.36,0.67,0.91


In [33]:
delays_middle_i_all_mag, delays_middle_e_all_mag = [], []
delays_middle_i_all_pzt, delays_middle_e_all_pzt = [], []

for activity in overview_middle_all_participants.keys():

    delays_middle_i_all_mag += overview_middle_all_participants[activity]["MAG"]["delay_i"] 
    delays_middle_e_all_mag += overview_middle_all_participants[activity]["MAG"]["delay_e"]

    delays_middle_i_all_pzt += overview_middle_all_participants[activity]["PZT"]["delay_i"]
    delays_middle_e_all_pzt += overview_middle_all_participants[activity]["PZT"]["delay_e"]

print(f"min delay mag {min(delays_middle_i_all_mag + delays_middle_e_all_mag, key=abs)}")
print(f"min delay pzt {min(delays_middle_i_all_pzt + delays_middle_e_all_pzt, key=abs)}")

print(f"max delay mag {max(delays_middle_i_all_mag + delays_middle_e_all_mag, key=abs)}")
print(f"max delay pzt {max(delays_middle_i_all_pzt + delays_middle_e_all_pzt, key=abs)}")

print(f"mean delay mag {round(np.mean(delays_middle_i_all_mag + delays_middle_e_all_mag), 2)} +/- {np.round(np.std(delays_middle_i_all_mag + delays_middle_e_all_mag), 2)}")
print(f"mean delay pzt {np.round(np.mean(delays_middle_i_all_pzt + delays_middle_e_all_pzt), 2)} +/- {np.round(np.std(delays_middle_i_all_pzt + delays_middle_e_all_pzt), 2)}")

min delay mag -0.0
min delay pzt -0.0
max delay mag 4.57
max delay pzt 4.68
mean delay mag 0.19 +/- 0.44
mean delay pzt 0.57 +/- 0.91
